In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Tải CatBoost
!pip install catboost
from catboost import CatBoostClassifier, Pool

# Tải pytorch-tabnet
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetClassifier

import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.5 MB/s eta 0:00:00


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Data/df_train.csv')
val = pd.read_csv('/content/drive/MyDrive/Data/df_validate.csv')

test_files = [f'/content/drive/MyDrive/Data/test_P{i}.csv' for i in range(1, 6)]
tests = {f: pd.read_csv(f) for f in test_files}

target_col = 'label'
static_drop_cols = ['label', 'user_id', 'course_id']

def get_clean_features(df):
  all_cols = df.columns.tolist()

  cols_to_drop = [c for c in all_cols if c in static_drop_cols]

  object_cols = df.select_dtypes(include=['object']).columns.tolist()

  print(f"Các cột Object sẽ bị loại bỏ: {len(object_cols)}")

  print(object_cols)

  cols_to_drop.extend(object_cols)

  cols_to_drop = list(set(cols_to_drop))

  final_features = [c for c in all_cols if c not in cols_to_drop]

  return final_features

features_to_use = get_clean_features(train)

print(f"\nTổng số features sử dụng: {len(features_to_use)}")
print(features_to_use)

Các cột Object sẽ bị loại bỏ: 30
['user_id', 'course_id', 'enroll_time', 'class_start', 'class_end', 'cutoff_time_P1', 'first_submit_time_P1', 'last_submit_time_P1', 'cutoff_time_P2', 'first_submit_time_P2', 'last_submit_time_P2', 'cutoff_time_P3', 'first_submit_time_P3', 'last_submit_time_P3', 'cutoff_time_P4', 'first_submit_time_P4', 'last_submit_time_P4', 'cutoff_time_P5', 'first_submit_time_P5', 'last_submit_time_P5', 'first_watch_time_P1', 'last_watch_time_P1', 'first_watch_time_P2', 'last_watch_time_P2', 'first_watch_time_P3', 'last_watch_time_P3', 'first_watch_time_P4', 'last_watch_time_P4', 'first_watch_time_P5', 'last_watch_time_P5']

Tổng số features sử dụng: 131
['class_duration_days', 'remaining_time', 'start_year', 'start_month', 'end_year', 'end_month', 'video_count', 'total_assignment_score', 'num_field', 'field_embed_0', 'field_embed_1', 'field_embed_2', 'field_embed_3', 'field_embed_4', 'field_embed_5', 'field_embed_6', 'field_embed_7', 'num_prerequisites', 'certificat

In [5]:
X_train = train[features_to_use]
y_train = train[target_col]

X_val = val[features_to_use]
y_val = val[target_col]

X_tests = {}
for file_path, df_test in tests.items():
  file_name = file_path.split('/')[-1]
  X_tests[file_name] = df_test[features_to_use]

print(f"Shape Train: {X_train.shape}")
print(f"Shape Val: {X_val.shape}")
print(f"Số lượng tập Test cần dự đoán: {len(X_tests)}")

Shape Train: (86632, 131)
Shape Val: (10829, 131)
Số lượng tập Test cần dự đoán: 5


In [6]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Đã cấu hình StratifiedKFold(n_splits=5, shuffle=True)")

Đã cấu hình StratifiedKFold(n_splits=5, shuffle=True)


In [10]:
print("=== Bắt đầu Grid Search cho CatBoost ===")

cb_model = CatBoostClassifier(
    iterations=250,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    verbose=0,
    early_stopping_rounds=50
)

param_grid_cb = {
    'depth': [4, 6],
    'learning_rate': [0.05],
}

grid_cb = GridSearchCV(
    estimator=cb_model,
    param_grid=param_grid_cb,
    cv=cv_strategy,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

grid_cb.fit(X_train, y_train)

print(f"\n>> Best CatBoost Params: {grid_cb.best_params_}")
print(f">> Best CV AUC: {grid_cb.best_score_:.4f}")

best_cb_model = grid_cb.best_estimator_

=== Bắt đầu Grid Search cho CatBoost ===
Fitting 5 folds for each of 2 candidates, totalling 10 fits

>> Best CatBoost Params: {'depth': 6, 'learning_rate': 0.05}
>> Best CV AUC: 0.9724


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

print("=== Bắt đầu KNN ===")

knn_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(weights='distance', n_jobs=-1)) # Fix weights='distance'
])

param_grid_knn_light = {
    'knn__n_neighbors': [30, 50]
}

grid_knn = GridSearchCV(
    estimator=knn_pipeline,
    param_grid=param_grid_knn_light,
    cv=cv_strategy,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

grid_knn.fit(X_train, y_train)

print(f">> Best KNN Params: {grid_knn.best_params_}")
print(f">> Best CV AUC: {grid_knn.best_score_:.4f}")
best_knn_model = grid_knn.best_estimator_

=== Bắt đầu KNN ===
Fitting 5 folds for each of 2 candidates, totalling 10 fits
>> Best KNN Params: {'knn__n_neighbors': 30}
>> Best CV AUC: 0.9458


In [ ]:
imputer_tn = SimpleImputer(strategy='mean')
X_train_np = imputer_tn.fit_transform(X_train)
y_train_np = y_train.values
X_val_np = imputer_tn.transform(X_val)
y_val_np = y_val.values

best_tn_model = TabNetClassifier(
    n_d=16, n_a=16,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":20, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax',
    verbose=1
)

best_tn_model.fit(
    X_train=X_train_np, y_train=y_train_np,
    eval_set=[(X_train_np, y_train_np), (X_val_np, y_val_np)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=30,
    patience=5,
    batch_size=1024,
    virtual_batch_size=128
)

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.33898 | train_auc: 0.71908 | valid_auc: 0.72677 |  0:00:07s
epoch 1  | loss: 0.24729 | train_auc: 0.69128 | valid_auc: 0.69212 |  0:00:13s
epoch 2  | loss: 0.20943 | train_auc: 0.65589 | valid_auc: 0.656   |  0:00:19s
epoch 3  | loss: 0.19268 | train_auc: 0.72657 | valid_auc: 0.73265 |  0:00:26s
epoch 4  | loss: 0.18104 | train_auc: 0.69851 | valid_auc: 0.7072  |  0:00:31s
epoch 5  | loss: 0.16913 | train_auc: 0.72089 | valid_auc: 0.72729 |  0:00:38s
epoch 6  | loss: 0.16258 | train_auc: 0.71871 | valid_auc: 0.71631 |  0:00:44s
epoch 7  | loss: 0.16189 | train_auc: 0.85831 | valid_auc: 0.85444 |  0:00:50s
epoch 8  | loss: 0.15388 | train_auc: 0.9437  | valid_auc: 0.94487 |  0:00:56s
epoch 9  | loss: 0.15203 | train_auc: 0.96344 | valid_auc: 0.96205 |  0:01:02s
epoch 10 | loss: 0.14832 | train_auc: 0.96479 | valid_auc: 0.96625 |  0:01:08s
epoch 11 | loss: 0.14272 | train_auc: 0.96695 | valid_auc: 0.96736 |  0:01:14s
epoch 12 | loss: 0.14131 | train_auc: 0.96617 | vali

In [14]:
print("=== Dự đoán cuối cùng ===")

for name, X_test in X_tests.items():
    print(f"Predicting {name}...")

    pred_cb = best_cb_model.predict_proba(X_test)[:, 1]
    pred_knn = best_knn_model.predict_proba(X_test)[:, 1]

    X_test_np = imputer_tn.transform(X_test)
    pred_tn = best_tn_model.predict_proba(X_test_np)[:, 1]

    final_pred = (0.6 * pred_cb) + (0.2 * pred_knn) + (0.2 * pred_tn)

    pd.DataFrame({
        'user_id': tests[f'/content/drive/MyDrive/Data/{name}']['user_id'],
        'label': final_pred
    }).to_csv(f"submission_light_{name}", index=False)

print("Xong!")

=== Dự đoán cuối cùng ===
Predicting test_P1.csv...
Predicting test_P2.csv...
Predicting test_P3.csv...
Predicting test_P4.csv...
Predicting test_P5.csv...
Xong!


In [15]:
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report

print("=== ĐÁNH GIÁ HIỆU SUẤT TRÊN TẬP TEST THỰC TẾ ===")

performance_summary = []

for name, X_test in X_tests.items():
    # 1. Lấy nhãn thực tế (Ground Truth)
    full_path = f'/content/drive/MyDrive/Data/{name}'

    if 'label' not in tests[full_path].columns:
        print(f"⚠️ {name}: Không tìm thấy cột label để đánh giá.")
        continue

    y_true = tests[full_path]['label']

    # 2. Tính toán lại xác suất dự đoán (Ensemble)
    pred_cb = best_cb_model.predict_proba(X_test)[:, 1]
    pred_knn = best_knn_model.predict_proba(X_test)[:, 1]

    X_test_np = imputer_tn.transform(X_test)
    pred_tn = best_tn_model.predict_proba(X_test_np)[:, 1]

    final_pred_prob = (0.6 * pred_cb) + (0.2 * pred_knn) + (0.2 * pred_tn)

    # Chuyển xác suất thành nhãn 0/1 (ngưỡng 0.5)
    final_pred_label = (final_pred_prob >= 0.5).astype(int)

    # 3. Tính Metrics
    auc = roc_auc_score(y_true, final_pred_prob)
    acc = accuracy_score(y_true, final_pred_label)

    print(f"\n>> Kết quả {name}:")
    print(f"   AUC: {auc:.4f}")
    print(f"   Accuracy: {acc:.4f}")

    # Lưu vào list để tổng kết
    performance_summary.append({'File': name, 'AUC': auc, 'Accuracy': acc})

# In bảng tổng kết
print("\n=== TỔNG KẾT ===")
summary_df = pd.DataFrame(performance_summary)
print(summary_df)

=== ĐÁNH GIÁ HIỆU SUẤT TRÊN TẬP TEST THỰC TẾ ===

>> Kết quả test_P1.csv:
   AUC: 0.8883
   Accuracy: 0.8993

>> Kết quả test_P2.csv:
   AUC: 0.8702
   Accuracy: 0.8958

>> Kết quả test_P3.csv:
   AUC: 0.9238
   Accuracy: 0.8965

>> Kết quả test_P4.csv:
   AUC: 0.9688
   Accuracy: 0.9285

>> Kết quả test_P5.csv:
   AUC: 0.9783
   Accuracy: 0.9562

=== TỔNG KẾT ===
          File       AUC  Accuracy
0  test_P1.csv  0.888306  0.899252
1  test_P2.csv  0.870217  0.895835
2  test_P3.csv  0.923753  0.896482
3  test_P4.csv  0.968789  0.928525
4  test_P5.csv  0.978312  0.956229


In [16]:
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd

print("=== BÁO CÁO HIỆU SUẤT CHI TIẾT ===")

# Danh sách lưu kết quả để tạo bảng báo cáo
report_data = []

for name, X_test in X_tests.items():
    # 1. Lấy nhãn thực tế (Ground Truth)
    full_path = f'/content/drive/MyDrive/Data/{name}'
    if 'label' not in tests[full_path].columns:
        continue
    y_true = tests[full_path]['label']

    # 2. Dự đoán riêng lẻ
    # CatBoost
    pred_cb = best_cb_model.predict_proba(X_test)[:, 1]

    # KNN
    pred_knn = best_knn_model.predict_proba(X_test)[:, 1]

    # TabNet (cần input numpy)
    X_test_np = imputer_tn.transform(X_test)
    pred_tn = best_tn_model.predict_proba(X_test_np)[:, 1]

    # 3. Ensemble (Kết hợp)
    pred_ensemble = (0.6 * pred_cb) + (0.2 * pred_knn) + (0.2 * pred_tn)

    # 4. Tính AUC cho từng model
    auc_cb = roc_auc_score(y_true, pred_cb)
    auc_knn = roc_auc_score(y_true, pred_knn)
    auc_tn = roc_auc_score(y_true, pred_tn)
    auc_ens = roc_auc_score(y_true, pred_ensemble)

    # Lưu vào danh sách
    report_data.append({
        'Test Set': name.replace('.csv', ''),
        'CatBoost AUC': auc_cb,
        'KNN AUC': auc_knn,
        'TabNet AUC': auc_tn,
        'Ensemble AUC': auc_ens
    })

# 5. Hiển thị bảng kết quả
df_report = pd.DataFrame(report_data)

df_report = df_report[['Test Set', 'CatBoost AUC', 'KNN AUC', 'TabNet AUC', 'Ensemble AUC']]

print("\n")
print(df_report.to_string(index=False))

# 6. Phân tích nhanh
print("\n--- NHẬN XÉT NHANH ---")
best_model = df_report[['CatBoost AUC', 'KNN AUC', 'TabNet AUC']].mean().idxmax()
print(f"🏆 Model đơn lẻ mạnh nhất trung bình: {best_model}")

avg_improvement = df_report['Ensemble AUC'].mean() - df_report[best_model].mean()
if avg_improvement > 0:
    print(f"Ensemble giúp tăng AUC trung bình thêm: +{avg_improvement:.4f}")
else:
    print(f"Ensemble đang thấp hơn {best_model}. Bạn nên xem lại trọng số!")

=== BÁO CÁO HIỆU SUẤT CHI TIẾT ===


Test Set  CatBoost AUC  KNN AUC  TabNet AUC  Ensemble AUC
 test_P1      0.819612 0.835840    0.881177      0.888306
 test_P2      0.770821 0.780309    0.878818      0.870217
 test_P3      0.827510 0.835748    0.926067      0.923753
 test_P4      0.964160 0.931484    0.942333      0.968789
 test_P5      0.975274 0.952221    0.973454      0.978312

--- NHẬN XÉT NHANH ---
🏆 Model đơn lẻ mạnh nhất trung bình: TabNet AUC
Ensemble giúp tăng AUC trung bình thêm: +0.0055
